09-homework

## Homework

In this homework, we'll deploy the Straight vs Curly Hair Type model we trained in the
[previous homework](../08-deep-learning/homework.md).

Download the model files from here: 

* https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
* https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx

With wget:

```bash
PREFIX="https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL="${PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL="${PREFIX}/hair_classifier_v1.onnx"
wget ${DATA_URL}
wget ${MODEL_URL}
```


In [15]:
import numpy as np
import os
import onnx
import onnxruntime as ort
import matplotlib.pyplot as plt


## Question 1

To be able to use this model, we need to know the name of the input and output nodes. 

What's the name of the output:

* `output`
* `sigmoid`
* `softmax`
* `prediction`

In [16]:

model = onnx.load("hair_classifier_v1.onnx")
graph = model.graph

print("Inputs:")
for inp in graph.input:
    print(f"name = {inp.name}")
    #print(inp.type)

print("\nOutputs:")
for out in graph.output:
    print(f"name = {out.name}")
    #print(out.type)


Inputs:
name = input

Outputs:
name = output


In [17]:

session = ort.InferenceSession("hair_classifier_v1.onnx")

print("Inputs:")
for i in session.get_inputs():
    print(f"name = {i.name}, type = {i.type}, shape = {i.shape}")

print("\nOutputs:")
for o in session.get_outputs():
    print(f"name = {o.name}, type = {o.type}, shape = {o.shape}")

Inputs:
name = input, type = tensor(float), shape = ['s77', 3, 200, 200]

Outputs:
name = output, type = tensor(float), shape = ['s77', 1]


In [18]:
session = ort.InferenceSession("hair_classifier_v1.onnx", providers=["CPUExecutionProvider"])

inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name

print(f"Input name: {input_name}")
print(f"Output name: {output_name}")

Input name: input
Output name: output


## Preparing the image

You'll need some code for downloading and resizing images. You can use 
this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have `pillow` installed:

```bash
pip install pillow
```

## Question 2: Target size

Let's download and resize this image: 

https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

Based on the previous homework, what should be the target size for the image?

* 64x64
* 128x128
* 200x200
* 256x256

In [19]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [20]:
print("Inputs:")
for i in session.get_inputs():
    print(f"name = {i.name}, type = {i.type}, shape = {i.shape}")

Inputs:
name = input, type = tensor(float), shape = ['s77', 3, 200, 200]


## Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* -10.73
* -1.073
* 1.073
* 10.73

In [21]:
download_image_url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"

img = download_image(download_image_url)

img = prepare_image(img, (200, 200))


In [22]:
from torchvision import transforms
val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [23]:
img_transform = val_transforms(img)

img_transform[0]

tensor([[-1.0733, -1.0048, -1.0390,  ..., -1.0733, -1.0733, -1.2103],
        [-1.0562, -1.0219, -1.0048,  ..., -1.0219, -1.0562, -1.1760],
        [-0.9534, -0.9705, -0.9192,  ..., -1.0219, -1.1075, -1.2274],
        ...,
        [-1.6727, -1.6727, -1.5185,  ...,  1.7352,  1.6495,  1.7865],
        [-1.6727, -1.6555, -1.6384,  ...,  1.6838,  1.5982,  1.7009],
        [-1.6384, -1.7240, -1.6727,  ...,  1.7180,  1.7352,  1.6838]])

In [52]:
def preprocess_pytorch(X):
    # X: shape (1, 299, 299, 3), dtype=float32, values in [0, 255]
    X = X / 255.0

    mean = np.array([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1)
    std = np.array([0.229, 0.224, 0.225]).reshape(1, 3, 1, 1)

    # Convert NHWC → NCHW
    # from (batch, height, width, channels) → (batch, channels, height, width)
    X = X.transpose(0, 3, 1, 2)

    # Normalize
    X = (X - mean) / std

    return X.astype(np.float32)


# Convert to numpy array, ensure float32
imgarr = np.array(img).astype(np.float32)  # shape (H, W, C)

# Add batch dimension → (1, H, W, C)
imgarr = np.expand_dims(imgarr, axis=0)
imgarr = preprocess_pytorch(imgarr)  # shape (1, C, H, W)
print(imgarr.shape)

(1, 3, 200, 200)


In [30]:

onnx_model_path = "hair_classifier_v1.onnx"
session = ort.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

In [ ]:
inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name

In [45]:
print(img_transform.shape)
print(img_transform.unsqueeze(0).shape)
print(img_transform.unsqueeze(0).cpu().numpy().shape)

torch.Size([3, 200, 200])
torch.Size([1, 3, 200, 200])
(1, 3, 200, 200)


In [54]:


ort_inputs = {session.get_inputs()[0].name: img_transform.unsqueeze(0).cpu().numpy()}
ort_outs = session.run(None, ort_inputs)


print(ort_outs)
logits = ort_outs[0][0][0]
print(logits)
prob = 1 / (1 + np.exp(-logits))
pred = 1 if prob > 0.5 else 0
print(f"Predicted class: {pred}, Probability: {prob:.4f}")

[array([[0.09156641]], dtype=float32)]
0.09156641
Predicted class: 1, Probability: 0.5229


In [55]:


ort_inputs = {session.get_inputs()[0].name: imgarr}
ort_outs = session.run(None, ort_inputs)


print(ort_outs)
logits = ort_outs[0][0][0]
print(logits)
prob = 1 / (1 + np.exp(-logits))
pred = 1 if prob > 0.5 else 0
print(f"Predicted class: {pred}, Probability: {prob:.4f}")

[array([[0.09156627]], dtype=float32)]
0.09156627
Predicted class: 1, Probability: 0.5229


## Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.09
* 0.49
* 0.69
* 0.89

## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file.  


## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.13

COPY hair_classifier_empty.onnx.data .
COPY hair_classifier_empty.onnx .
```

Note that it uses Python 3.13.

The docker image is published to [`agrigorev/model-2024-hairstyle:v3`](https://hub.docker.com/r/agrigorev/model-2024-hairstyle/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.

## Question 5

Download the base image `agrigorev/model-2025-hairstyle:v1`. You can do it with [`docker pull`](https://docs.docker.com/engine/reference/commandline/pull/).

So what's the size of this base image?

* 88 Mb
* 208 Mb
* 608 Mb
* 1208 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.


REPOSITORY                       TAG       IMAGE ID       CREATED       SIZE
agrigorev/model-2025-hairstyle   v1        4528ad1525d5   2 days ago    608MB

## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `hair_classifier_empty.onnx` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference). 
The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

What's the output from the model?

* -1.0
* -0.10
* 0.10
* 1.0

## Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout 
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.


## Submit the results

* Submit your results here: https://courses.datatalks.club/ml-zoomcamp-2025/homework/hw09
* If your answer doesn't match options exactly, select the closest one. If the answer is exactly in between two options, select the higher value.

## Publishing to Docker hub

Just for the reference, this is how we published our image to Docker hub:

```bash
docker build -t model-2025-hairstyle -f homework.dockerfile .
docker tag model-2025-hairstyle:latest agrigorev/model-2025-hairstyle:v1
docker push agrigorev/model-2025-hairstyle:v1
```

(You don't need to execute this code)